<a href="https://colab.research.google.com/github/DGkurnia/sentiment/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Nama : Derfansyah Guswiranata Kurnia
- Iddicoding: dgkurnia
- email dicoding: weerakurnia@gmai.com

#Insiasi

In [1]:
#Perpustakaan tahap awal
!pip install selenium beautifulsoup4 pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 20.6 MB/s eta 0:00:00


In [26]:
#Impor Bagian awal
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
#modifikasi percepatan
import asyncio
import aiohttp
from concurrent.futures import ProcessPoolExecutor

In [3]:
#Next Import
import datetime as dt  # Manipulasi data waktu dan tanggal
import re  # Modul untuk bekerja dengan ekspresi reguler
import string  # Berisi konstanta string, seperti tanda baca
from nltk.tokenize import word_tokenize  # Tokenisasi teks
from nltk.corpus import stopwords  # Daftar kata-kata berhenti dalam teks

In [4]:
#Tambahan
from selenium.webdriver.chrome.service import Service

## Sastrawi

In [5]:
#Instalasi sastrawi
!pip install sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.4 MB/s eta 0:00:00


In [6]:
#Impor Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory  # Stemming (penghilangan imbuhan kata) dalam bahasa Indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory  # Menghapus kata-kata berhenti dalam bahasa Indonesia
#ke wordcloud

In [27]:
#seksi wordcloud
!pip install wordcloud
from wordcloud import WordCloud  # Membuat visualisasi berbentuk awan kata (word cloud) dari teks
# ke NITK

In [28]:
#NITK
import nltk  # Import pustaka NLTK (Natural Language Toolkit).
nltk.download('punkt')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stopwords) dalam berbagai bahasa.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [29]:
#impor file tambahan (google drive)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Perpustakaan lain untuk analisis

In [22]:
#Seksi Io dan lain-lain
import os
from io import StringIO
import concurrent.futures #untuk pemrograman paralel
from concurrent.futures import ThreadPoolExecutor

In [8]:
#Interpolasi untuk perbaikan: Grafik
import numpy as np
import matplotlib.pyplot as plt
from multiprocessing import Pool
#perpustakaan untuk interpolasi
from scipy.interpolate import interp1d

In [9]:
#Bagian lain
import csv
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score, f1_score

### Persiapan untuk analisis

In [25]:
#pandas
import pandas as pd
import time

In [10]:
#Ekstraksi
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.sentiment import SentimentIntensityAnalyzer  #sentimen
#pemisahan
from sklearn.model_selection import train_test_split as ttsplit
#ke klasifikasi

In [11]:
#klasifikasi randomforest
from sklearn.ensemble import RandomForestClassifier as rfclasifi #teknik 'randomfprest'
from sklearn.tree import DecisionTreeClassifier #teknik 'keputusan'
from sklearn.svm import SVC #teknik SVC
from sklearn.linear_model import LogisticRegression

## Pengambilan ulasan dari situs Tokopedia

In [32]:
#Tahap awal pengambilan
urllist = [
    "https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-jazlyn-black-jazlyn-black-s/review",
    "https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-gribson-rayon-navy-s/review",
    "https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-lavrenti-white-lavrenti-white-s/review"
]

Ada tiga ulasan yang dipakai untuk analisis

In [33]:
#Persiapan untuk proses pengambilan list: Tahap Extraksi HTM secara asinkronus
async def fetch_html(url):
    loop = asyncio.get_event_loop()
    response = await loop.run_in_executor(None, requests.get, url)
    return response.content

# Fungsi ekstraksi secara asinkronus
async def review_scrapper(url):
    html_content = await fetch_html(url)
    soup = BeautifulSoup(html_content, 'html.parser')

    # Ekstraksi ulasan
    ulasan = []
    for rev in soup.find_all('div', class_='review-class'):
        teks = rev.find('div', class_='review-content').text.strip()
        ulasan.append(teks)

    return ulasan
#ke potongan

In [34]:
#persiapan asinkronus untuk pemrosesan url
def process_urls(chunk):
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

    # Gather results from the asynchronous review scrapper
    tasks = [review_scrapper(url) for url in chunk]
    results = loop.run_until_complete(asyncio.gather(*tasks))

    # Flatten the list of results
    return [item for sublist in results for item in sublist]
#ke fungsi utama

Pengambilan nilai ulasan per potongan

In [35]:
#fungsi pemotongan analisis
def chunk_and_execute(urls, chunk_size=10, output_prefix='intermediate_reviews'):
    totulas = []

    # Inspeksi intermediet
    existing_files = [f for f in os.listdir() if f.startswith(output_prefix) and f.endswith('.csv')]
    last_processed_index = len(existing_files) - 1 if existing_files else -1

    # Mulai dari titik persimpanan terbaru
    for i in range(last_processed_index + 1):
        df = pd.read_csv(f'{output_prefix}{i}.csv')
        totulas.extend(df['Review'].tolist())

    # Start processing from the next chunk
    num_chunks = (len(urls) + chunk_size - 1) // chunk_size  # Ceiling division

    with ProcessPoolExecutor() as executor:
        for i in range(last_processed_index + 1, num_chunks):
            start_idx = i * chunk_size
            end_idx = min((i + 1) * chunk_size, len(urls))

            chunk_url_list = urls[start_idx:end_idx]

            # Process the current chunk of URLs using multiprocessing
            results = executor.submit(process_urls, chunk_url_list).result()

            # Extend the main list with scraped data
            totulas.extend(results)

            # Save intermediate results to CSV
            pd.DataFrame(totulas[:], columns=['Review']).to_csv(f'{output_prefix}{i}.csv', index=False)

    return totulas
#ke tahap akhir

In [ ]:
#penyatuan ke hasil akhir
hslakhir = chunk_and_execute(urllist)
#ke penyetelan data csv

totulas adalah **'total ulasan'** yang dipakai untuk analisis hasi akhir **(hslakhir)**

In [ ]:
#persiapan df untuk pembersihan
dfori = pd.DataFrame(hslakhir, columns=['Review'])
dfkopi = dfori.copy() #Jangan pakai data asli
dfkopi.to_csv('tokopedia_reviews.csv', index=False)

data yang dipakai adala **'dfkopi'**

In [ ]:
#Inspeksi dataset
dfinfo = dfkopi.info()
print(dfinfo)

# Tahap awal (pembersihan)

In [ ]:
#persiapan untuk pembersihan


In [ ]:
#Persiapan tahap awal
